In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# # source: https://github.com/nielsdenissen/qlearning/blob/master/qlearning.py
# class Qlearning:
#     _qmatrix = None
#     _learn_rate = None
#     _discount_factor = None

#     def __init__(self,
#                  possible_states,
#                  possible_actions,
#                  initial_reward,
#                  learning_rate,
#                  discount_factor):
#         """
#         Initialise the q learning class with an initial matrix and the parameters for learning.
#         :param possible_states: list of states the agent can be in
#         :param possible_actions: list of actions the agent can perform
#         :param initial_reward: the initial Q-values to be used in the matrix
#         :param learning_rate: the learning rate used for Q-learning
#         :param discount_factor: the discount factor used for Q-learning
#         """
#         # Initialize the matrix with Q-values
#         init_data = [[float(initial_reward) for _ in possible_states]
#                      for _ in possible_actions]
#         self._qmatrix = pd.DataFrame(data=init_data,
#                                      index=possible_actions,
#                                      columns=possible_states)

#         # Save the parameters
#         self._learn_rate = learning_rate
#         self._discount_factor = discount_factor

#     def get_best_action(self, state):
#         """
#         Retrieve the action resulting in the highest Q-value for a given state.
#         :param state: the state for which to determine the best action
#         :return: the best action from the given state
#         """
#         # Return the action (index) with maximum Q-value
#         return self._qmatrix[[state]].idxmax().iloc[0]

#     def update_model(self, state, action, reward, next_state):
#         """
#         Update the Q-values for a given observation.
#         :param state: The state the observation started in
#         :param action: The action taken from that state
#         :param reward: The reward retrieved from taking action from state
#         :param next_state: The resulting next state of taking action from state
#         """
#         # Update q_value for a state-action pair Q(s,a):
#         # Q(s,a) = Q(s,a) + α( r + γmaxa' Q(s',a') - Q(s,a) )
#         q_sa = self._qmatrix.ix[action, state]
#         max_q_sa_next = self._qmatrix.ix[self.get_best_action(next_state), next_state]
#         r = reward
#         alpha = self._learn_rate
#         gamma = self._discount_factor

#         # Do the computation
#         new_q_sa = q_sa + alpha * (r + gamma * max_q_sa_next - q_sa)
#         self._qmatrix.set_value(action, state, new_q_sa)

In [3]:
# class for all states participating in the ventilator distribution project
class States(object):
#     class_var = 1
    def __init__(self, num_ventilators_req, num_venti_have):
        self.vr = num_ventilators_req # number of ventilators required when the optimization begins
        self.vh = num_venti_have # number of ventilators already present in the pile

In [4]:
# set up variables
fed_pile = 13000 # ventilators in the federal stockpile
days = 100 # total days you want to run the tool for
n_states = 4 # total number of states participating
state_to_state = np.zeros((n_states,n_states)) # transfer of ventilators between states

In [5]:
# new ventilators that are arriving due to purchases and new manufacturing
incoming_ventilators = np.zeros(100)
for i in range(0,100):
    incoming_ventilators[i] = i*5

In [6]:
# Create state variables
California = States(1300,500)
New_York = States(10000,5550)
Illinois = States(250, 5000)
Florida = States(1200, 1500)

In [7]:
# array to store ventilators needed for every state
vent_needed = [California.vr, New_York.vr, Illinois.vr, Florida.vr]
vent_needed = np.asarray(vent_needed)

In [8]:
# array to store ventilators available for every state
vent_available = [California.vh, New_York.vh, Illinois.vh, Florida.vh]
vent_available = np.asarray(vent_available)

In [9]:
def next_state1(vent_needed, vent_available, state_to_state, n_states):
    vent_needed_next = np.copy(vent_needed)
    vent_available_next = np.copy(vent_available)
    for i in range(0, n_states):
        vent_needed_next[i] = vent_needed[i] - np.sum(state_to_state[:,i]) + np.sum(state_to_state[i,:])
        vent_available_next[i] = vent_available[i] + np.sum(state_to_state[:,i]) - np.sum(state_to_state[i,:])
    return vent_needed_next, vent_available_next

In [10]:
def get_reward(vent_available, vent_needed_in_three_days):
    reward = vent_available - vent_needed_in_three_days
    return reward

In [11]:
# https://github.com/dennybritz/reinforcement-learning/blob/master/DP/README.md

In [12]:
# l = 0
# for i in range(0,4):
#     for j in range(0,4):
#         state_to_state[i,j] = l
#         l = l + 1
# state_to_state[0,0] = state_to_state[1,1] = state_to_state[2,2] = state_to_state[3,3] = 0

In [13]:
# next_state(vent_needed, vent_available, state_to_state, n_states)

In [14]:
def max_allocate_first(vent_needed, vent_available, n_states):
    state_to_state = np.zeros((n_states,n_states)) # state_to_state transfer matrix
    vent_actual_available = []
    vent_actual_needed = []
    # find how many ventilators each state actually needs and how many are available
    for i in range(0,len(vent_needed)):
        vent_actual_available.append(vent_available[i] - vent_needed[i])
        vent_actual_needed.append(vent_needed[i] - vent_available[i])
    vent_actual_available = np.asarray(vent_actual_available)
    vent_actual_needed = np.asarray(vent_actual_needed)
    print("vent_actual_available: ", vent_actual_available)
    print("vent_actual_needed: ", vent_actual_needed)
    ranked = np.argsort(vent_actual_needed) # sort the ventilators needed array in asceding order
    largest_indices = ranked[::-1][:len(vent_actual_needed)] # change ascending to descending order
    print("largest_indices: ", largest_indices)
    
    total_ventilators_available = vent_actual_available[vent_actual_available > 0].sum()
    total_ventilators_needed = vent_actual_needed[vent_actual_needed > 0].sum()
    
    print("==========================")
    for arg in largest_indices:
        print(arg)
        current_allocator = -1
        print(vent_actual_needed[arg] > 0 and total_ventilators_available > 0)
        if (vent_actual_needed[arg] <= 0):
            continue
        elif (vent_actual_needed[arg] > 0 and total_ventilators_available > 0):
            while (total_ventilators_available > 0 or vent_actual_needed[arg] != 0) and (vent_actual_needed[arg] > 0):
                if vent_actual_available[largest_indices[current_allocator]] > 0:
                    if vent_actual_available[largest_indices[current_allocator]] > vent_actual_needed[arg]:
                        vent_actual_available[largest_indices[current_allocator]] = vent_actual_available[largest_indices[current_allocator]] - vent_actual_needed[arg]
                        total_ventilators_available = vent_actual_available[vent_actual_available > 0].sum()
                        state_to_state[largest_indices[current_allocator], arg] = vent_actual_needed[arg]
                        vent_actual_needed[arg] = 0
                        total_ventilators_needed = vent_actual_needed[vent_actual_needed > 0].sum()
                    elif vent_actual_available[largest_indices[current_allocator]] < vent_actual_needed[arg]:
                        vent_actual_needed[arg] = vent_actual_needed[arg] - vent_actual_available[largest_indices[current_allocator]]
                        state_to_state[largest_indices[current_allocator], arg] = vent_actual_available[largest_indices[current_allocator]] 
                        vent_actual_available[largest_indices[current_allocator]] = 0
                        total_ventilators_available = vent_actual_available[vent_actual_available > 0].sum()
                        total_ventilators_needed = vent_actual_needed[vent_actual_needed > 0].sum()
                        current_allocator = current_allocator - 1
                        if abs(current_allocator) > abs(len(vent_actual_needed)):
                            break
                else:
                    current_allocator = current_allocator - 1
                    if abs(current_allocator) > abs(len(vent_actual_needed)):
                            break
                print("vent_actual_available: ", vent_actual_available)
                print("vent_actual_needed: ", vent_actual_needed)
                print("total_ventilators_available: ", total_ventilators_available)
                print("total_ventilators_needed: ", total_ventilators_needed)
                print("==========================")
                if total_ventilators_available <= 0:
                    break
#     return state_to_state
    return (state_to_state)

In [15]:
def max_allocate_first_limit(vent_needed, vent_available, n_states, state_allocation_limit):
    state_to_state = np.zeros((n_states,n_states)) # state_to_state transfer matrix
    vent_actual_available = []
    vent_actual_needed = []
    # find how many ventilators each state actually needs and how many are available
    for i in range(0,len(vent_needed)):
        vent_actual_available.append(vent_available[i] - vent_needed[i])
        vent_actual_needed.append(vent_needed[i] - vent_available[i])
    vent_actual_available = np.asarray(vent_actual_available)
    vent_actual_needed = np.asarray(vent_actual_needed)
    print("vent_actual_available: ", vent_actual_available)
    print("vent_actual_needed: ", vent_actual_needed)
    ranked = np.argsort(vent_actual_needed) # sort the ventilators needed array in asceding order
    largest_indices = ranked[::-1][:len(vent_actual_needed)] # change ascending to descending order
    print("largest_indices: ", largest_indices)
    
    total_ventilators_available = vent_actual_available[vent_actual_available > 0].sum()
    total_ventilators_needed = vent_actual_needed[vent_actual_needed > 0].sum()
    flag = np.zeros(n_states)
    
    print("==========================")
    for arg in largest_indices:
        print(arg)
        current_allocator = -1
        print(vent_actual_needed[arg] > 0 and total_ventilators_available > 0)
        if (vent_actual_needed[arg] <= 0):
            continue
        elif (vent_actual_needed[arg] > 0 and total_ventilators_available > 0):
            while (total_ventilators_available > 0 or vent_actual_needed[arg] != 0) and (np.count_nonzero(flag) < n_states) and (vent_actual_needed[arg] > 0):
                if vent_actual_available[largest_indices[current_allocator]] > 0:
                    if vent_actual_available[largest_indices[current_allocator]] > vent_actual_needed[arg]:
                        vent_actual_available[largest_indices[current_allocator]] = vent_actual_available[largest_indices[current_allocator]] - vent_actual_needed[arg]
                        total_ventilators_available = vent_actual_available[vent_actual_available > 0].sum()
                        if vent_actual_needed[arg] > state_allocation_limit[largest_indices[current_allocator]]:
                            state_to_state[largest_indices[current_allocator], arg] = state_allocation_limit[largest_indices[current_allocator]]
                            flag[largest_indices[current_allocator]] = 1
                        elif vent_actual_needed[arg] < state_allocation_limit[largest_indices[current_allocator]]:
                            state_to_state[largest_indices[current_allocator], arg] = vent_actual_needed[arg]
                        vent_actual_needed[arg] = 0
                        total_ventilators_needed = vent_actual_needed[vent_actual_needed > 0].sum()
                    elif vent_actual_available[largest_indices[current_allocator]] < vent_actual_needed[arg]:
                        vent_actual_needed[arg] = vent_actual_needed[arg] - vent_actual_available[largest_indices[current_allocator]]
                        if vent_actual_needed[arg] > state_allocation_limit[largest_indices[current_allocator]]:
                            state_to_state[largest_indices[current_allocator], arg] = state_allocation_limit[largest_indices[current_allocator]]
                            flag[largest_indices[current_allocator]] = 1
                        elif vent_actual_needed[arg] < state_allocation_limit[largest_indices[current_allocator]]:
                            state_to_state[largest_indices[current_allocator], arg] = vent_actual_needed[arg] 
                        vent_actual_available[largest_indices[current_allocator]] = 0
                        total_ventilators_available = vent_actual_available[vent_actual_available > 0].sum()
                        total_ventilators_needed = vent_actual_needed[vent_actual_needed > 0].sum()
                        current_allocator = current_allocator - 1
                        if abs(current_allocator) > abs(len(vent_actual_needed)):
                            break
                else:
                    current_allocator = current_allocator - 1
                    if abs(current_allocator) > abs(len(vent_actual_needed)):
                            break
                print("vent_actual_available: ", vent_actual_available)
                print("vent_actual_needed: ", vent_actual_needed)
                print("total_ventilators_available: ", total_ventilators_available)
                print("total_ventilators_needed: ", total_ventilators_needed)
                print("==========================")
                if total_ventilators_available <= 0:
                    break
#     return state_to_state
    return (state_to_state)

In [79]:
def max_allocate_first_fed(vent_needed, vent_available, n_states, state_allocation_limit, fed_pile, fed_pile_limit):
    state_to_state = np.zeros((n_states+1,n_states+1)) # state_to_state transfer matrix
    vent_actual_available = []
    vent_actual_needed = []
    # find how many ventilators each state actually needs and how many are available
    for i in range(0,len(vent_needed)):
        vent_actual_available.append(vent_available[i] - vent_needed[i])
        vent_actual_needed.append(vent_needed[i] - vent_available[i])
    vent_actual_available = np.asarray(vent_actual_available)
    vent_actual_needed = np.asarray(vent_actual_needed)
    print("vent_actual_available: ", vent_actual_available)
    print("vent_actual_needed: ", vent_actual_needed)
    ranked = np.argsort(vent_actual_needed) # sort the ventilators needed array in asceding order
    largest_indices = ranked[::-1][:len(vent_actual_needed)] # change ascending to descending order
    print("largest_indices: ", largest_indices)
    
    total_ventilators_available = vent_actual_available[vent_actual_available > 0].sum()
    total_ventilators_needed = vent_actual_needed[vent_actual_needed > 0].sum()
    flag = np.zeros(n_states)
    
    print("==========================")
    for arg in largest_indices:
        fed_flag = 0
        print(arg)
        current_allocator = -1
        print(vent_actual_needed[arg] > 0 and total_ventilators_available > 0)
        if (vent_actual_needed[arg] <= 0):
            continue
        elif (vent_actual_needed[arg] > 0 and total_ventilators_available > 0):
            while (total_ventilators_available > 0 or vent_actual_needed[arg] != 0) and (np.count_nonzero(flag) < n_states) and (vent_actual_needed[arg] > 0):
                if vent_actual_available[largest_indices[current_allocator]] > 0:
                    if vent_actual_available[largest_indices[current_allocator]] > vent_actual_needed[arg]:
                        vent_actual_available[largest_indices[current_allocator]] = vent_actual_available[largest_indices[current_allocator]] - vent_actual_needed[arg]
                        total_ventilators_available = vent_actual_available[vent_actual_available > 0].sum()
                        if vent_actual_needed[arg] > state_allocation_limit[largest_indices[current_allocator]]:
                            state_to_state[largest_indices[current_allocator], arg] = state_allocation_limit[largest_indices[current_allocator]]
                            if fed_flag == 0 and fed_pile > fed_pile_limit:
                                if vent_actual_needed[arg] > fed_pile_limit:
                                    state_to_state[n_states, arg] = fed_pile_limit
                                    fed_flag = 1
                                    fed_pile = fed_pile - fed_pile_limit
                                elif vent_actual_needed[arg] < fed_pile_limit:
                                    state_to_state[n_states, arg] = vent_actual_needed[arg]
                                    fed_flag = 1
                                    fed_pile = fed_pile - vent_actual_needed[arg]
                            if fed_flag == 0 and fed_pile < fed_pile_limit and fed_pile > 0:
                                if vent_actual_needed[arg] > fed_pile:
                                    state_to_state[n_states, arg] = fed_pile
                                    fed_flag = 1
                                    fed_pile = 0
                                elif vent_actual_needed[arg] < fed_pile:
                                    state_to_state[n_states, arg] = vent_actual_needed[arg]
                                    fed_flag = 1
                                    fed_pile = fed_pile - vent_actual_needed[arg]
                            flag[largest_indices[current_allocator]] = 1
                        elif vent_actual_needed[arg] < state_allocation_limit[largest_indices[current_allocator]]:
                            state_to_state[largest_indices[current_allocator], arg] = vent_actual_needed[arg]
                        vent_actual_needed[arg] = 0
                        total_ventilators_needed = vent_actual_needed[vent_actual_needed > 0].sum()
                    elif vent_actual_available[largest_indices[current_allocator]] < vent_actual_needed[arg]:
                        vent_actual_needed[arg] = vent_actual_needed[arg] - vent_actual_available[largest_indices[current_allocator]]
                        if vent_actual_needed[arg] > state_allocation_limit[largest_indices[current_allocator]]:
                            state_to_state[largest_indices[current_allocator], arg] = state_allocation_limit[largest_indices[current_allocator]]
                            flag[largest_indices[current_allocator]] = 1
                            if fed_flag == 0 and fed_pile > fed_pile_limit:
                                if vent_actual_needed[arg] > fed_pile_limit:
                                    state_to_state[n_states, arg] = fed_pile_limit
                                    fed_flag = 1
                                    fed_pile = fed_pile - fed_pile_limit
                                elif vent_actual_needed[arg] < fed_pile_limit:
                                    state_to_state[n_states, arg] = vent_actual_needed[arg]
                                    fed_flag = 1
                                    fed_pile = fed_pile - vent_actual_needed[arg]
                            if fed_flag == 0 and fed_pile < fed_pile_limit and fed_pile > 0:
                                if vent_actual_needed[arg] > fed_pile:
                                    state_to_state[n_states, arg] = fed_pile
                                    fed_flag = 1
                                    fed_pile = 0
                                elif vent_actual_needed[arg] < fed_pile:
                                    state_to_state[n_states, arg] = vent_actual_needed[arg]
                                    fed_flag = 1
                                    fed_pile = fed_pile - vent_actual_needed[arg]
                        elif vent_actual_needed[arg] < state_allocation_limit[largest_indices[current_allocator]]:
                            state_to_state[largest_indices[current_allocator], arg] = vent_actual_needed[arg] 
                        vent_actual_available[largest_indices[current_allocator]] = 0
                        total_ventilators_available = vent_actual_available[vent_actual_available > 0].sum()
                        total_ventilators_needed = vent_actual_needed[vent_actual_needed > 0].sum()
                        current_allocator = current_allocator - 1
                        if abs(current_allocator) > abs(len(vent_actual_needed)):
                            break
                else:
                    current_allocator = current_allocator - 1
                    if abs(current_allocator) > abs(len(vent_actual_needed)):
                            break
                print("vent_actual_available: ", vent_actual_available)
                print("vent_actual_needed: ", vent_actual_needed)
                print("total_ventilators_available: ", total_ventilators_available)
                print("total_ventilators_needed: ", total_ventilators_needed)
                print("==========================")
                if total_ventilators_available <= 0:
                    break
#     return state_to_state
    return (state_to_state, fed_pile)

In [75]:
aa = max_allocate_first(vent_needed, vent_available, n_states)
print(aa)

vent_actual_available:  [ -800 -4450  4750   300]
vent_actual_needed:  [  800  4450 -4750  -300]
largest_indices:  [1 0 3 2]
1
True
vent_actual_available:  [ -800 -4450   300   300]
vent_actual_needed:  [  800     0 -4750  -300]
total_ventilators_available:  600
total_ventilators_needed:  800
0
True
vent_actual_available:  [ -800 -4450     0   300]
vent_actual_needed:  [  500     0 -4750  -300]
total_ventilators_available:  300
total_ventilators_needed:  500
vent_actual_available:  [ -800 -4450     0     0]
vent_actual_needed:  [  200     0 -4750  -300]
total_ventilators_available:  0
total_ventilators_needed:  200
3
False
2
False
[[   0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.]
 [ 300. 4450.    0.    0.    0.]
 [ 300.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.]]


In [76]:
vent_needed1 = [1300, 10000, 250, 1200, 0]

In [77]:
vent_available1 = [ 500, 5550, 5000, 1500, 13000]

In [78]:
aa = max_allocate_first_fed(vent_needed, vent_available, 5, [100,25,50,100], 13000, 500)
print(aa)

vent_actual_available:  [ -800 -4450  4750   300]
vent_actual_needed:  [  800  4450 -4750  -300]
largest_indices:  [1 0 3 2]
1
True
vent_actual_available:  [ -800 -4450   300   300]
vent_actual_needed:  [  800     0 -4750  -300]
total_ventilators_available:  600
total_ventilators_needed:  800
0
True
vent_actual_available:  [ -800 -4450     0   300]
vent_actual_needed:  [  500     0 -4750  -300]
total_ventilators_available:  300
total_ventilators_needed:  500
vent_actual_available:  [ -800 -4450     0     0]
vent_actual_needed:  [  200     0 -4750  -300]
total_ventilators_available:  0
total_ventilators_needed:  200
3
False
2
False
[[  0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [ 50.  50.   0.   0.   0.   0.]
 [100.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [200. 500.   0.   0.   0.   0.]]


In [18]:
def min_allocate_first(vent_needed, vent_available, n_states):
    state_to_state = np.zeros((n_states,n_states)) # state_to_state transfer matrix
    vent_actual_available = []
    vent_actual_needed = []
    # find how many ventilators each state actually needs and how many are available
    for i in range(0,len(vent_needed)):
        vent_actual_available.append(vent_available[i] - vent_needed[i])
        vent_actual_needed.append(vent_needed[i] - vent_available[i])
    vent_actual_available = np.asarray(vent_actual_available)
    vent_actual_needed = np.asarray(vent_actual_needed)
    print("vent_actual_available: ", vent_actual_available)
    print("vent_actual_needed: ", vent_actual_needed)
    largest_indices = np.argsort(vent_actual_needed) # sort the ventilators needed array in asceding order
    print("largest_indices: ", largest_indices)
    
    total_ventilators_available = vent_actual_available[vent_actual_available > 0].sum()
    total_ventilators_needed = vent_actual_needed[vent_actual_needed > 0].sum()
    
    print("==========================")
    for arg in largest_indices:
        print(arg)
        current_allocator = -1
        print(vent_actual_needed[arg] > 0 and total_ventilators_available > 0)
        if (vent_actual_needed[arg] <= 0):
            continue
        elif (vent_actual_needed[arg] > 0 and total_ventilators_available > 0):
            while (total_ventilators_available > 0 or vent_actual_needed[arg] != 0) and (vent_actual_needed[arg] > 0):
                if vent_actual_available[largest_indices[current_allocator]] > 0:
                    if vent_actual_available[largest_indices[current_allocator]] > vent_actual_needed[arg]:
                        vent_actual_available[largest_indices[current_allocator]] = vent_actual_available[largest_indices[current_allocator]] - vent_actual_needed[arg]
                        total_ventilators_available = vent_actual_available[vent_actual_available > 0].sum()
                        state_to_state[largest_indices[current_allocator], arg] = vent_actual_needed[arg]
                        vent_actual_needed[arg] = 0
                        total_ventilators_needed = vent_actual_needed[vent_actual_needed > 0].sum()
                    elif vent_actual_available[largest_indices[current_allocator]] < vent_actual_needed[arg]:
                        vent_actual_needed[arg] = vent_actual_needed[arg] - vent_actual_available[largest_indices[current_allocator]]
                        state_to_state[largest_indices[current_allocator], arg] = vent_actual_available[largest_indices[current_allocator]] 
                        vent_actual_available[largest_indices[current_allocator]] = 0
                        total_ventilators_available = vent_actual_available[vent_actual_available > 0].sum()
                        total_ventilators_needed = vent_actual_needed[vent_actual_needed > 0].sum()
                        current_allocator = current_allocator - 1
                        if abs(current_allocator) > abs(len(vent_actual_needed)):
                            break
                else:
                    current_allocator = current_allocator - 1
                    if abs(current_allocator) > abs(len(vent_actual_needed)):
                            break
                print("vent_actual_available: ", vent_actual_available)
                print("vent_actual_needed: ", vent_actual_needed)
                print("total_ventilators_available: ", total_ventilators_available)
                print("total_ventilators_needed: ", total_ventilators_needed)
                print("==========================")
                if total_ventilators_available <= 0:
                    break
#     return state_to_state
    return (state_to_state)

In [19]:
def min_allocate_first_limit(vent_needed, vent_available, n_states, state_allocation_limit):
    state_to_state = np.zeros((n_states,n_states)) # state_to_state transfer matrix
    vent_actual_available = []
    vent_actual_needed = []
    # find how many ventilators each state actually needs and how many are available
    for i in range(0,len(vent_needed)):
        vent_actual_available.append(vent_available[i] - vent_needed[i])
        vent_actual_needed.append(vent_needed[i] - vent_available[i])
    vent_actual_available = np.asarray(vent_actual_available)
    vent_actual_needed = np.asarray(vent_actual_needed)
    print("vent_actual_available: ", vent_actual_available)
    print("vent_actual_needed: ", vent_actual_needed)
    largest_indices = np.argsort(vent_actual_needed) # sort the ventilators needed array in asceding order
    print("largest_indices: ", largest_indices)
    
    total_ventilators_available = vent_actual_available[vent_actual_available > 0].sum()
    total_ventilators_needed = vent_actual_needed[vent_actual_needed > 0].sum()
    flag = np.zeros(n_states)
    
    print("==========================")
    for arg in largest_indices:
        print(arg)
        current_allocator = -1
        print(vent_actual_needed[arg] > 0 and total_ventilators_available > 0)
        if (vent_actual_needed[arg] <= 0):
            continue
        elif (vent_actual_needed[arg] > 0 and total_ventilators_available > 0):
            while (total_ventilators_available > 0 or vent_actual_needed[arg] != 0)  and (np.count_nonzero(flag) < n_states) and (vent_actual_needed[arg] > 0):
                if vent_actual_available[largest_indices[current_allocator]] > 0:
                    if vent_actual_available[largest_indices[current_allocator]] > vent_actual_needed[arg]:
                        vent_actual_available[largest_indices[current_allocator]] = vent_actual_available[largest_indices[current_allocator]] - vent_actual_needed[arg]
                        total_ventilators_available = vent_actual_available[vent_actual_available > 0].sum()
                        if vent_actual_needed[arg] > state_allocation_limit[largest_indices[current_allocator]]:
                            state_to_state[largest_indices[current_allocator], arg] = state_allocation_limit[largest_indices[current_allocator]]
                            flag[largest_indices[current_allocator]] = 1
                        elif vent_actual_needed[arg] < state_allocation_limit[largest_indices[current_allocator]]:
                            state_to_state[largest_indices[current_allocator], arg] = vent_actual_needed[arg]
                        vent_actual_needed[arg] = 0
                        total_ventilators_needed = vent_actual_needed[vent_actual_needed > 0].sum()
                    elif vent_actual_available[largest_indices[current_allocator]] < vent_actual_needed[arg]:
                        vent_actual_needed[arg] = vent_actual_needed[arg] - vent_actual_available[largest_indices[current_allocator]]
                        if vent_actual_needed[arg] > state_allocation_limit[largest_indices[current_allocator]]:
                            state_to_state[largest_indices[current_allocator], arg] = state_allocation_limit[largest_indices[current_allocator]]
                            flag[largest_indices[current_allocator]] = 1
                        elif vent_actual_needed[arg] < state_allocation_limit[largest_indices[current_allocator]]:
                            state_to_state[largest_indices[current_allocator], arg] = vent_actual_needed[arg] 
                        vent_actual_available[largest_indices[current_allocator]] = 0
                        total_ventilators_available = vent_actual_available[vent_actual_available > 0].sum()
                        total_ventilators_needed = vent_actual_needed[vent_actual_needed > 0].sum()
                        current_allocator = current_allocator - 1
                        if abs(current_allocator) > abs(len(vent_actual_needed)):
                            break
                else:
                    current_allocator = current_allocator - 1
                    if abs(current_allocator) > abs(len(vent_actual_needed)):
                            break
                print("vent_actual_available: ", vent_actual_available)
                print("vent_actual_needed: ", vent_actual_needed)
                print("total_ventilators_available: ", total_ventilators_available)
                print("total_ventilators_needed: ", total_ventilators_needed)
                print("==========================")
                if total_ventilators_available <= 0:
                    break
#     return state_to_state
    return (state_to_state)

In [88]:
def min_allocate_first_fed(vent_needed, vent_available, n_states, state_allocation_limit, fed_pile, fed_pile_limit):
    state_to_state = np.zeros((n_states+1,n_states+1)) # state_to_state transfer matrix
    vent_actual_available = []
    vent_actual_needed = []
    # find how many ventilators each state actually needs and how many are available
    for i in range(0,len(vent_needed)):
        vent_actual_available.append(vent_available[i] - vent_needed[i])
        vent_actual_needed.append(vent_needed[i] - vent_available[i])
    vent_actual_available = np.asarray(vent_actual_available)
    vent_actual_needed = np.asarray(vent_actual_needed)
    print("vent_actual_available: ", vent_actual_available)
    print("vent_actual_needed: ", vent_actual_needed)
    largest_indices = np.argsort(vent_actual_needed) # sort the ventilators needed array in asceding order
    print("largest_indices: ", largest_indices)
    
    total_ventilators_available = vent_actual_available[vent_actual_available > 0].sum()
    total_ventilators_needed = vent_actual_needed[vent_actual_needed > 0].sum()
    flag = np.zeros(n_states)
    
    print("==========================")
    for arg in largest_indices:
        fed_flag = 0
        print(arg)
        current_allocator = -1
        print(vent_actual_needed[arg] > 0 and total_ventilators_available > 0)
        if (vent_actual_needed[arg] <= 0):
            continue
        elif (vent_actual_needed[arg] > 0 and total_ventilators_available > 0):
            while (total_ventilators_available > 0 or vent_actual_needed[arg] != 0)  and (np.count_nonzero(flag) < n_states) and (vent_actual_needed[arg] > 0):
                if vent_actual_available[largest_indices[current_allocator]] > 0:
                    if vent_actual_available[largest_indices[current_allocator]] > vent_actual_needed[arg]:
                        vent_actual_available[largest_indices[current_allocator]] = vent_actual_available[largest_indices[current_allocator]] - vent_actual_needed[arg]
                        total_ventilators_available = vent_actual_available[vent_actual_available > 0].sum()
                        if vent_actual_needed[arg] > state_allocation_limit[largest_indices[current_allocator]]:
                            state_to_state[largest_indices[current_allocator], arg] = state_allocation_limit[largest_indices[current_allocator]]
                            flag[largest_indices[current_allocator]] = 1
                            if fed_flag == 0 and fed_pile > fed_pile_limit:
                                if vent_actual_needed[arg] > fed_pile_limit:
                                    state_to_state[n_states, arg] = fed_pile_limit
                                    fed_flag = 1
                                    fed_pile = fed_pile - fed_pile_limit
                                elif vent_actual_needed[arg] < fed_pile_limit:
                                    state_to_state[n_states, arg] = vent_actual_needed[arg]
                                    fed_flag = 1
                                    fed_pile = fed_pile - vent_actual_needed[arg]
                            if fed_flag == 0 and fed_pile < fed_pile_limit and fed_pile > 0:
                                if vent_actual_needed[arg] > fed_pile:
                                    state_to_state[n_states, arg] = fed_pile
                                    fed_flag = 1
                                    fed_pile = 0
                                elif vent_actual_needed[arg] < fed_pile:
                                    state_to_state[n_states, arg] = vent_actual_needed[arg]
                                    fed_flag = 1
                                    fed_pile = fed_pile - vent_actual_needed[arg]
                        elif vent_actual_needed[arg] < state_allocation_limit[largest_indices[current_allocator]]:
                            state_to_state[largest_indices[current_allocator], arg] = vent_actual_needed[arg]
                        vent_actual_needed[arg] = 0
                        total_ventilators_needed = vent_actual_needed[vent_actual_needed > 0].sum()
                    elif vent_actual_available[largest_indices[current_allocator]] < vent_actual_needed[arg]:
                        vent_actual_needed[arg] = vent_actual_needed[arg] - vent_actual_available[largest_indices[current_allocator]]
                        if vent_actual_needed[arg] > state_allocation_limit[largest_indices[current_allocator]]:
                            state_to_state[largest_indices[current_allocator], arg] = state_allocation_limit[largest_indices[current_allocator]]
                            flag[largest_indices[current_allocator]] = 1
                            if fed_flag == 0 and fed_pile > fed_pile_limit:
                                if vent_actual_needed[arg] > fed_pile_limit:
                                    state_to_state[n_states, arg] = fed_pile_limit
                                    fed_flag = 1
                                    fed_pile = fed_pile - fed_pile_limit
                                elif vent_actual_needed[arg] < fed_pile_limit:
                                    state_to_state[n_states, arg] = vent_actual_needed[arg]
                                    fed_flag = 1
                                    fed_pile = fed_pile - vent_actual_needed[arg]
                            if fed_flag == 0 and fed_pile < fed_pile_limit and fed_pile > 0:
                                if vent_actual_needed[arg] > fed_pile:
                                    state_to_state[n_states, arg] = fed_pile
                                    fed_flag = 1
                                    fed_pile = 0
                                elif vent_actual_needed[arg] < fed_pile:
                                    state_to_state[n_states, arg] = vent_actual_needed[arg]
                                    fed_flag = 1
                                    fed_pile = fed_pile - vent_actual_needed[arg]
                        elif vent_actual_needed[arg] < state_allocation_limit[largest_indices[current_allocator]]:
                            state_to_state[largest_indices[current_allocator], arg] = vent_actual_needed[arg] 
                        vent_actual_available[largest_indices[current_allocator]] = 0
                        total_ventilators_available = vent_actual_available[vent_actual_available > 0].sum()
                        total_ventilators_needed = vent_actual_needed[vent_actual_needed > 0].sum()
                        current_allocator = current_allocator - 1
                        if abs(current_allocator) > abs(len(vent_actual_needed)):
                            break
                else:
                    current_allocator = current_allocator - 1
                    if abs(current_allocator) > abs(len(vent_actual_needed)):
                            break
                print("vent_actual_available: ", vent_actual_available)
                print("vent_actual_needed: ", vent_actual_needed)
                print("total_ventilators_available: ", total_ventilators_available)
                print("total_ventilators_needed: ", total_ventilators_needed)
                print("==========================")
                if total_ventilators_available <= 0:
                    break
#     return state_to_state
    return (state_to_state, fed_pile)

In [89]:
aa = min_allocate_first(vent_needed, vent_available, n_states)
print(aa)

vent_actual_available:  [ -800 -4450  4750   300]
vent_actual_needed:  [  800  4450 -4750  -300]
largest_indices:  [2 3 0 1]
2
False
3
False
0
True
vent_actual_available:  [ -800 -4450  4750   300]
vent_actual_needed:  [  800  4450 -4750  -300]
total_ventilators_available:  5050
total_ventilators_needed:  5250
vent_actual_available:  [ -800 -4450  4750   300]
vent_actual_needed:  [  800  4450 -4750  -300]
total_ventilators_available:  5050
total_ventilators_needed:  5250
vent_actual_available:  [ -800 -4450  4750     0]
vent_actual_needed:  [  500  4450 -4750  -300]
total_ventilators_available:  4750
total_ventilators_needed:  4950
vent_actual_available:  [ -800 -4450  4250     0]
vent_actual_needed:  [    0  4450 -4750  -300]
total_ventilators_available:  4250
total_ventilators_needed:  4450
1
True
vent_actual_available:  [ -800 -4450  4250     0]
vent_actual_needed:  [    0  4450 -4750  -300]
total_ventilators_available:  4250
total_ventilators_needed:  4450
vent_actual_available:  [

In [90]:
aa = min_allocate_first_fed(vent_needed, vent_available, n_states, [100,25,50,100], 13000, 400)
print(aa)

vent_actual_available:  [ -800 -4450  4750   300]
vent_actual_needed:  [  800  4450 -4750  -300]
largest_indices:  [2 3 0 1]
2
False
3
False
0
True
vent_actual_available:  [ -800 -4450  4750   300]
vent_actual_needed:  [  800  4450 -4750  -300]
total_ventilators_available:  5050
total_ventilators_needed:  5250
vent_actual_available:  [ -800 -4450  4750   300]
vent_actual_needed:  [  800  4450 -4750  -300]
total_ventilators_available:  5050
total_ventilators_needed:  5250
vent_actual_available:  [ -800 -4450  4750     0]
vent_actual_needed:  [  500  4450 -4750  -300]
total_ventilators_available:  4750
total_ventilators_needed:  4950
vent_actual_available:  [ -800 -4450  4250     0]
vent_actual_needed:  [    0  4450 -4750  -300]
total_ventilators_available:  4250
total_ventilators_needed:  4450
1
True
vent_actual_available:  [ -800 -4450  4250     0]
vent_actual_needed:  [    0  4450 -4750  -300]
total_ventilators_available:  4250
total_ventilators_needed:  4450
vent_actual_available:  [

In [22]:
def random_allocate(vent_needed, vent_available, n_states):
    state_to_state = np.zeros((n_states,n_states)) # state_to_state transfer matrix
    vent_actual_available = []
    vent_actual_needed = []
    # find how many ventilators each state actually needs and how many are available
    for i in range(0,len(vent_needed)):
        vent_actual_available.append(vent_available[i] - vent_needed[i])
        vent_actual_needed.append(vent_needed[i] - vent_available[i])
    vent_actual_available = np.asarray(vent_actual_available)
    vent_actual_needed = np.asarray(vent_actual_needed)
    print("vent_actual_available: ", vent_actual_available)
    print("vent_actual_needed: ", vent_actual_needed)
    largest_indices = np.argsort(vent_actual_needed) # sort the ventilators needed array in asceding order
    print("largest_indices: ", largest_indices)
    
    total_ventilators_available = vent_actual_available[vent_actual_available > 0].sum()
    total_ventilators_needed = vent_actual_needed[vent_actual_needed > 0].sum()
    np.random.shuffle(largest_indices)
    print("==========================")
    for arg in largest_indices:
        print(arg)
        current_allocator = -1
        print(vent_actual_needed[arg] > 0 and total_ventilators_available > 0)
        if (vent_actual_needed[arg] <= 0):
            continue
        elif (vent_actual_needed[arg] > 0 and total_ventilators_available > 0):
            while (total_ventilators_available > 0 or vent_actual_needed[arg] != 0) and (vent_actual_needed[arg] > 0):
                if vent_actual_available[largest_indices[current_allocator]] > 0:
                    if vent_actual_available[largest_indices[current_allocator]] > vent_actual_needed[arg]:
                        vent_actual_available[largest_indices[current_allocator]] = vent_actual_available[largest_indices[current_allocator]] - vent_actual_needed[arg]
                        total_ventilators_available = vent_actual_available[vent_actual_available > 0].sum()
                        state_to_state[largest_indices[current_allocator], arg] = vent_actual_needed[arg]
                        vent_actual_needed[arg] = 0
                        total_ventilators_needed = vent_actual_needed[vent_actual_needed > 0].sum()
                    elif vent_actual_available[largest_indices[current_allocator]] < vent_actual_needed[arg]:
                        vent_actual_needed[arg] = vent_actual_needed[arg] - vent_actual_available[largest_indices[current_allocator]]
                        state_to_state[largest_indices[current_allocator], arg] = vent_actual_available[largest_indices[current_allocator]] 
                        vent_actual_available[largest_indices[current_allocator]] = 0
                        total_ventilators_available = vent_actual_available[vent_actual_available > 0].sum()
                        total_ventilators_needed = vent_actual_needed[vent_actual_needed > 0].sum()
                        current_allocator = current_allocator - 1
                        if abs(current_allocator) > abs(len(vent_actual_needed)):
                            break
                else:
                    current_allocator = current_allocator - 1
                    if abs(current_allocator) > abs(len(vent_actual_needed)):
                            break
                print("vent_actual_available: ", vent_actual_available)
                print("vent_actual_needed: ", vent_actual_needed)
                print("total_ventilators_available: ", total_ventilators_available)
                print("total_ventilators_needed: ", total_ventilators_needed)
                print("==========================")
                if total_ventilators_available <= 0:
                    break
#     return state_to_state
    return (state_to_state)

In [23]:
def random_allocate_limit(vent_needed, vent_available, n_states, state_allocation_limit):
    state_to_state = np.zeros((n_states,n_states)) # state_to_state transfer matrix
    vent_actual_available = []
    vent_actual_needed = []
    # find how many ventilators each state actually needs and how many are available
    for i in range(0,len(vent_needed)):
        vent_actual_available.append(vent_available[i] - vent_needed[i])
        vent_actual_needed.append(vent_needed[i] - vent_available[i])
    vent_actual_available = np.asarray(vent_actual_available)
    vent_actual_needed = np.asarray(vent_actual_needed)
    print("vent_actual_available: ", vent_actual_available)
    print("vent_actual_needed: ", vent_actual_needed)
    largest_indices = np.argsort(vent_actual_needed) # sort the ventilators needed array in asceding order
    print("largest_indices: ", largest_indices)
    
    total_ventilators_available = vent_actual_available[vent_actual_available > 0].sum()
    total_ventilators_needed = vent_actual_needed[vent_actual_needed > 0].sum()
    np.random.shuffle(largest_indices)
    flag = np.zeros(n_states)
    
    print("==========================")
    for arg in largest_indices:
        print(arg)
        current_allocator = -1
        print(vent_actual_needed[arg] > 0 and total_ventilators_available > 0)
        if (vent_actual_needed[arg] <= 0):
            continue
        elif (vent_actual_needed[arg] > 0 and total_ventilators_available > 0):
            while (total_ventilators_available > 0 or vent_actual_needed[arg] != 0) and (np.count_nonzero(flag) < n_states) and (vent_actual_needed[arg] > 0):
                if vent_actual_available[largest_indices[current_allocator]] > 0:
                    if vent_actual_available[largest_indices[current_allocator]] > vent_actual_needed[arg]:
                        vent_actual_available[largest_indices[current_allocator]] = vent_actual_available[largest_indices[current_allocator]] - vent_actual_needed[arg]
                        total_ventilators_available = vent_actual_available[vent_actual_available > 0].sum()
                        if vent_actual_needed[arg] > state_allocation_limit[largest_indices[current_allocator]]:
                            state_to_state[largest_indices[current_allocator], arg] = state_allocation_limit[largest_indices[current_allocator]]
                            flag[largest_indices[current_allocator]] = 1
                        elif vent_actual_needed[arg] < state_allocation_limit[largest_indices[current_allocator]]:
                            state_to_state[largest_indices[current_allocator], arg] = vent_actual_needed[arg]
                        vent_actual_needed[arg] = 0
                        total_ventilators_needed = vent_actual_needed[vent_actual_needed > 0].sum()
                    elif vent_actual_available[largest_indices[current_allocator]] < vent_actual_needed[arg]:
                        vent_actual_needed[arg] = vent_actual_needed[arg] - vent_actual_available[largest_indices[current_allocator]]
                        if vent_actual_needed[arg] > state_allocation_limit[largest_indices[current_allocator]]:
                            state_to_state[largest_indices[current_allocator], arg] = state_allocation_limit[largest_indices[current_allocator]]
                            flag[largest_indices[current_allocator]] = 1
                        elif vent_actual_needed[arg] < state_allocation_limit[largest_indices[current_allocator]]:
                            state_to_state[largest_indices[current_allocator], arg] = vent_actual_needed[arg] 
                        vent_actual_available[largest_indices[current_allocator]] = 0
                        total_ventilators_available = vent_actual_available[vent_actual_available > 0].sum()
                        total_ventilators_needed = vent_actual_needed[vent_actual_needed > 0].sum()
                        current_allocator = current_allocator - 1
                        if abs(current_allocator) > abs(len(vent_actual_needed)):
                            break
                else:
                    current_allocator = current_allocator - 1
                    if abs(current_allocator) > abs(len(vent_actual_needed)):
                            break
                print("vent_actual_available: ", vent_actual_available)
                print("vent_actual_needed: ", vent_actual_needed)
                print("total_ventilators_available: ", total_ventilators_available)
                print("total_ventilators_needed: ", total_ventilators_needed)
                print("==========================")
                if total_ventilators_available <= 0:
                    break
#     return state_to_state
    return (state_to_state)

In [84]:
def random_allocate_fed(vent_needed, vent_available, n_states, state_allocation_limit, fed_pile, fed_pile_limit):
    state_to_state = np.zeros((n_states+1,n_states+1)) # state_to_state transfer matrix
    vent_actual_available = []
    vent_actual_needed = []
    # find how many ventilators each state actually needs and how many are available
    for i in range(0,len(vent_needed)):
        vent_actual_available.append(vent_available[i] - vent_needed[i])
        vent_actual_needed.append(vent_needed[i] - vent_available[i])
    vent_actual_available = np.asarray(vent_actual_available)
    vent_actual_needed = np.asarray(vent_actual_needed)
    print("vent_actual_available: ", vent_actual_available)
    print("vent_actual_needed: ", vent_actual_needed)
    largest_indices = np.argsort(vent_actual_needed) # sort the ventilators needed array in asceding order
    print("largest_indices: ", largest_indices)
    
    total_ventilators_available = vent_actual_available[vent_actual_available > 0].sum()
    total_ventilators_needed = vent_actual_needed[vent_actual_needed > 0].sum()
    np.random.shuffle(largest_indices)
    flag = np.zeros(n_states)
    
    print("==========================")
    for arg in largest_indices:
        fed_flag = 0
        print(arg)
        current_allocator = -1
        print(vent_actual_needed[arg] > 0 and total_ventilators_available > 0)
        if (vent_actual_needed[arg] <= 0):
            continue
        elif (vent_actual_needed[arg] > 0 and total_ventilators_available > 0):
            while (total_ventilators_available > 0 or vent_actual_needed[arg] != 0) and (np.count_nonzero(flag) < n_states) and (vent_actual_needed[arg] > 0):
                if vent_actual_available[largest_indices[current_allocator]] > 0:
                    if vent_actual_available[largest_indices[current_allocator]] > vent_actual_needed[arg]:
                        vent_actual_available[largest_indices[current_allocator]] = vent_actual_available[largest_indices[current_allocator]] - vent_actual_needed[arg]
                        total_ventilators_available = vent_actual_available[vent_actual_available > 0].sum()
                        if vent_actual_needed[arg] > state_allocation_limit[largest_indices[current_allocator]]:
                            state_to_state[largest_indices[current_allocator], arg] = state_allocation_limit[largest_indices[current_allocator]]
                            flag[largest_indices[current_allocator]] = 1
                            if fed_flag == 0 and fed_pile > fed_pile_limit:
                                if vent_actual_needed[arg] > fed_pile_limit:
                                    state_to_state[n_states, arg] = fed_pile_limit
                                    fed_flag = 1
                                    fed_pile = fed_pile - fed_pile_limit
                                elif vent_actual_needed[arg] < fed_pile_limit:
                                    state_to_state[n_states, arg] = vent_actual_needed[arg]
                                    fed_flag = 1
                                    fed_pile = fed_pile - vent_actual_needed[arg]
                            if fed_flag == 0 and fed_pile < fed_pile_limit and fed_pile > 0:
                                if vent_actual_needed[arg] > fed_pile:
                                    state_to_state[n_states, arg] = fed_pile
                                    fed_flag = 1
                                    fed_pile = 0
                                elif vent_actual_needed[arg] < fed_pile:
                                    state_to_state[n_states, arg] = vent_actual_needed[arg]
                                    fed_flag = 1
                                    fed_pile = fed_pile - vent_actual_needed[arg]
                        elif vent_actual_needed[arg] < state_allocation_limit[largest_indices[current_allocator]]:
                            state_to_state[largest_indices[current_allocator], arg] = vent_actual_needed[arg]
                        vent_actual_needed[arg] = 0
                        total_ventilators_needed = vent_actual_needed[vent_actual_needed > 0].sum()
                    elif vent_actual_available[largest_indices[current_allocator]] < vent_actual_needed[arg]:
                        vent_actual_needed[arg] = vent_actual_needed[arg] - vent_actual_available[largest_indices[current_allocator]]
                        if vent_actual_needed[arg] > state_allocation_limit[largest_indices[current_allocator]]:
                            state_to_state[largest_indices[current_allocator], arg] = state_allocation_limit[largest_indices[current_allocator]]
                            flag[largest_indices[current_allocator]] = 1
                            if fed_flag == 0 and fed_pile > fed_pile_limit:
                                if vent_actual_needed[arg] > fed_pile_limit:
                                    state_to_state[n_states, arg] = fed_pile_limit
                                    fed_flag = 1
                                    fed_pile = fed_pile - fed_pile_limit
                                elif vent_actual_needed[arg] < fed_pile_limit:
                                    state_to_state[n_states, arg] = vent_actual_needed[arg]
                                    fed_flag = 1
                                    fed_pile = fed_pile - vent_actual_needed[arg]
                            if fed_flag == 0 and fed_pile < fed_pile_limit and fed_pile > 0:
                                if vent_actual_needed[arg] > fed_pile:
                                    state_to_state[n_states, arg] = fed_pile
                                    fed_flag = 1
                                    fed_pile = 0
                                elif vent_actual_needed[arg] < fed_pile:
                                    state_to_state[n_states, arg] = vent_actual_needed[arg]
                                    fed_flag = 1
                                    fed_pile = fed_pile - vent_actual_needed[arg]
                        elif vent_actual_needed[arg] < state_allocation_limit[largest_indices[current_allocator]]:
                            state_to_state[largest_indices[current_allocator], arg] = vent_actual_needed[arg] 
                        vent_actual_available[largest_indices[current_allocator]] = 0
                        total_ventilators_available = vent_actual_available[vent_actual_available > 0].sum()
                        total_ventilators_needed = vent_actual_needed[vent_actual_needed > 0].sum()
                        current_allocator = current_allocator - 1
                        if abs(current_allocator) > abs(len(vent_actual_needed)):
                            break
                else:
                    current_allocator = current_allocator - 1
                    if abs(current_allocator) > abs(len(vent_actual_needed)):
                            break
                print("vent_actual_available: ", vent_actual_available)
                print("vent_actual_needed: ", vent_actual_needed)
                print("total_ventilators_available: ", total_ventilators_available)
                print("total_ventilators_needed: ", total_ventilators_needed)
                print("==========================")
                if total_ventilators_available <= 0:
                    break
#     return state_to_state
    return (state_to_state, fed_pile)

In [85]:
aa = random_allocate(vent_needed, vent_available, n_states)
print(aa)

vent_actual_available:  [ -800 -4450  4750   300]
vent_actual_needed:  [  800  4450 -4750  -300]
largest_indices:  [2 3 0 1]
1
True
vent_actual_available:  [ -800 -4450   300   300]
vent_actual_needed:  [  800     0 -4750  -300]
total_ventilators_available:  600
total_ventilators_needed:  800
3
False
0
True
vent_actual_available:  [ -800 -4450     0   300]
vent_actual_needed:  [  500     0 -4750  -300]
total_ventilators_available:  300
total_ventilators_needed:  500
vent_actual_available:  [ -800 -4450     0   300]
vent_actual_needed:  [  500     0 -4750  -300]
total_ventilators_available:  300
total_ventilators_needed:  500
vent_actual_available:  [ -800 -4450     0     0]
vent_actual_needed:  [  200     0 -4750  -300]
total_ventilators_available:  0
total_ventilators_needed:  200
2
False
[[   0.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.]
 [ 300. 4450.    0.    0.    0.]
 [ 300.    0.    0.    0.    0.]
 [   0.    0.    0.    0.    0.]]


In [86]:
aa = random_allocate_fed(vent_needed, vent_available, n_states, [100,25,50,100], 13000, 400)
print(aa)

vent_actual_available:  [ -800 -4450  4750   300]
vent_actual_needed:  [  800  4450 -4750  -300]
largest_indices:  [2 3 0 1]
0
True
vent_actual_available:  [ -800 -4450  4750   300]
vent_actual_needed:  [  800  4450 -4750  -300]
total_ventilators_available:  5050
total_ventilators_needed:  5250
vent_actual_available:  [ -800 -4450  3950   300]
vent_actual_needed:  [    0  4450 -4750  -300]
total_ventilators_available:  4250
total_ventilators_needed:  4450
3
False
2
False
1
True
vent_actual_available:  [ -800 -4450  3950   300]
vent_actual_needed:  [    0  4450 -4750  -300]
total_ventilators_available:  4250
total_ventilators_needed:  4450
vent_actual_available:  [ -800 -4450     0   300]
vent_actual_needed:  [    0   500 -4750  -300]
total_ventilators_available:  300
total_ventilators_needed:  500
vent_actual_available:  [ -800 -4450     0     0]
vent_actual_needed:  [    0   200 -4750  -300]
total_ventilators_available:  0
total_ventilators_needed:  200
(array([[  0.,   0.,   0.,   0.

In [26]:
next_num1, next_num2 = next_state1(vent_needed, vent_available, aa, 4)


In [27]:
vent_needed, next_num1

(array([ 1300, 10000,   250,  1200]), array([1150, 9950,  350, 1300]))

In [28]:
aaa = random_allocate(next_num1, next_num2, n_states)

vent_actual_available:  [ -500 -4350  4550   100]
vent_actual_needed:  [  500  4350 -4550  -100]
largest_indices:  [2 3 0 1]
0
True
vent_actual_available:  [ -500 -4350  4550     0]
vent_actual_needed:  [  400  4350 -4550  -100]
total_ventilators_available:  4550
total_ventilators_needed:  4750
vent_actual_available:  [ -500 -4350  4150     0]
vent_actual_needed:  [    0  4350 -4550  -100]
total_ventilators_available:  4150
total_ventilators_needed:  4350
1
True
vent_actual_available:  [ -500 -4350  4150     0]
vent_actual_needed:  [    0  4350 -4550  -100]
total_ventilators_available:  4150
total_ventilators_needed:  4350
vent_actual_available:  [ -500 -4350     0     0]
vent_actual_needed:  [    0   200 -4550  -100]
total_ventilators_available:  0
total_ventilators_needed:  200
2
False
3
False


In [29]:
print(aaa)

[[   0.    0.    0.    0.]
 [   0.    0.    0.    0.]
 [ 400. 4150.    0.    0.]
 [ 100.    0.    0.    0.]]


In [30]:
vent_needed

array([ 1300, 10000,   250,  1200])

In [31]:
vent_available

array([ 500, 5550, 5000, 1500])

In [32]:
w = 0.1
discount_factor = 0.8
threshold = 0
max_allocate_in_one_step = 100
n_states = 5
n = 20
max_vent_requirement = 6000
ratio_step = 0.1

In [33]:
def next_state(venti_avail, venti_need):
    state = venti_avail/venti_needed
    return state

In [34]:
def find_where_it_fits(ratio, max_states, increment):
    max_val = increment*max_states
    min_val = 0
    i = 0
    ratio_steps = []
    while i < max_states:
        ratio_steps.append(i*increment)
        i = i + 1
    ratio_steps = np.asarray(ratio_steps)
    min_diff = float('inf')
    index = -1
    for j in range(0,max_states):
        diff = abs(ratio - ratio_steps[j])
        if diff < min_diff:
            min_diff = diff
            index = j
    return index

In [35]:
def create_reward(max_n, n):
    reward_state = np.zeros(n)
    for i in range(0,20):
        reward_state[i] = 0.1*i*max_n - max_n
        if reward_state[i] > 0:
            reward_state[i] = 0
    return reward_state

In [125]:
def create_reward_proxy(population_density, n):
    #https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States_by_population_density#2015_density_(states,_territories_and_DC)
    reward_state = np.zeros(n)
    for i in range(0,20):
        reward_state[i] = 0.1*i*population_density - population_density
        if reward_state[i] > 0:
            reward_state[i] = 0
    return reward_state

In [126]:
# create_reward_proxy

# California = 251
reward_state1 = create_reward_proxy(251, 20)

# New_York = 420
reward_state2 = create_reward_proxy(420, 20)

# Illinois = 231
reward_state3 = create_reward_proxy(231, 20)

# Florida = 378
reward_state4 = create_reward_proxy(378, 20)

# Kentucky = 112
reward_state5 = create_reward_proxy(112, 20)

# Kansas = 36
reward_state6 = create_reward_proxy(36, 20)

# Montana = 7
reward_state7 = create_reward_proxy(7, 20)

# Oregon = 41
reward_state8 = create_reward_proxy(41, 20)

# Oklahoma = 57
reward_state9 = create_reward_proxy(57, 20)

# New_Jersey = 1218
reward_state10 = create_reward_proxy(1218, 20)

# Massachusetts = 871
reward_state11 = create_reward_proxy(871, 20)

In [36]:
# California = 683
reward_state1 = create_reward(683, 20)

#New_York = 5789
reward_state2 = create_reward(5789, 20)

# Illinois = 945
reward_state3 = create_reward(945, 20)

# Florida = 401
reward_state4 = create_reward(401, 20)

# Kentucky = 90
reward_state5 = create_reward(90, 20)

# Kansas = 42
reward_state6 = create_reward(42, 20)

# Montana = 7
reward_state7 = create_reward(7, 20)

# Oregon = 28
reward_state8 = create_reward(28, 20)

# Oklahoma = 62
reward_state9 = create_reward(62, 20)

# New Jersey = 2381
reward_state10 = create_reward(2381, 20)

# Massachusetts = 1069
reward_state11 = create_reward(1069, 20)

In [37]:
reward_state2

array([-5789. , -5210.1, -4631.2, -4052.3, -3473.4, -2894.5, -2315.6,
       -1736.7, -1157.8,  -578.9,     0. ,     0. ,     0. ,     0. ,
           0. ,     0. ,     0. ,     0. ,     0. ,     0. ])

In [38]:
reward_state1

array([-683. , -614.7, -546.4, -478.1, -409.8, -341.5, -273.2, -204.9,
       -136.6,  -68.3,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,
          0. ,    0. ,    0. ,    0. ])

In [39]:
def optimal_value_iteration(max_vent_requirement, max_allocate_in_one_step, n, ratio_step, reward):
    i = 0
    vent_steps = []
    while i < max_vent_requirement/max_allocate_in_one_step:
        vent_steps.append(i*max_allocate_in_one_step)
        i = i + 1
    vent_steps = np.asarray(vent_steps)
    i = 0
    ratio_steps = []
    while i < n:
        ratio_steps.append(i*ratio_step)
        i = i + 1
    ratio_steps = np.asarray(ratio_steps)
    values = np.zeros((n))
    delta = float('inf')
    while delta > threshold:
        delta = 0
        for j in range(0, n):
            v = values[j]
            if j == 0:
                reward_backward = reward[j]
                val_backward = values[j]
                reward_stay = reward[j]
                reward_forward = reward[j+1]
                val_backward = values[j]
                val_stay = values[j]
                val_forward = values[j+1]
            elif j == n-1:
                reward_backward = reward[j-1]
                reward_stay = reward[j]
                reward_forward = reward[j]
                val_backward = values[j-1]
                val_stay = values[j]
                val_forward = values[j]
            else:
                reward_backward = reward[j-1]
                reward_stay = reward[j]
                reward_forward = reward[j+1]
                val_backward = values[j-1]
                val_stay = values[j]
                val_forward = values[j+1]
            total_reward_backward = reward_backward + discount_factor*val_backward
            total_reward_forward = reward_forward + discount_factor*val_forward
            total_reward_stay = reward_stay + discount_factor*val_stay
            values[j] = max(total_reward_backward, total_reward_stay, total_reward_forward)
            delta = max(delta, abs(v - values[j]))
    return(values)

In [40]:
vals1 = optimal_value_iteration(max_vent_requirement, max_allocate_in_one_step, n, ratio_step, reward_state1)
vals2 = optimal_value_iteration(max_vent_requirement, max_allocate_in_one_step, n, ratio_step, reward_state2)
vals3 = optimal_value_iteration(max_vent_requirement, max_allocate_in_one_step, n, ratio_step, reward_state3)
vals4 = optimal_value_iteration(max_vent_requirement, max_allocate_in_one_step, n, ratio_step, reward_state4)

In [41]:
vals1

array([-1890.84141645, -1595.17677056, -1310.9709632 , -1041.088704  ,
        -789.11088   ,  -559.5136    ,  -357.892     ,  -191.24      ,
         -68.3       ,     0.        ,     0.        ,     0.        ,
           0.        ,     0.        ,     0.        ,     0.        ,
           0.        ,     0.        ,     0.        ,     0.        ])

In [42]:
reward_state1

array([-683. , -614.7, -546.4, -478.1, -409.8, -341.5, -273.2, -204.9,
       -136.6,  -68.3,    0. ,    0. ,    0. ,    0. ,    0. ,    0. ,
          0. ,    0. ,    0. ,    0. ])

In [43]:
def optimal_policy(venti_available, venti_required, max_states, increment, max_allocate_in_one_step, n_states, vals1, vals2, vals3, vals4, state_allocation_limit):
    state_to_state = np.zeros((n_states,n_states)) # state_to_state transfer matrix
    state_state_1 = venti_available[0]/venti_required[0]
    state_state_2 = venti_available[1]/venti_required[1]
    state_state_3 = venti_available[2]/venti_required[2]
    state_state_4 = venti_available[3]/venti_required[3]
    where_state_1 = find_where_it_fits(state_state_1, max_states, increment)
    where_state_2 = find_where_it_fits(state_state_2, max_states, increment)
    where_state_3 = find_where_it_fits(state_state_3, max_states, increment)
    where_state_4 = find_where_it_fits(state_state_4, max_states, increment)
    where_state_1_give = find_where_it_fits((venti_available[0]-max_allocate_in_one_step)/venti_required[0], max_states, increment)
    where_state_2_give = find_where_it_fits((venti_available[1]-max_allocate_in_one_step)/venti_required[1], max_states, increment)
    where_state_3_give = find_where_it_fits((venti_available[2]-max_allocate_in_one_step)/venti_required[2], max_states, increment)
    where_state_4_give = find_where_it_fits((venti_available[3]-max_allocate_in_one_step)/venti_required[3], max_states, increment)
    where_state_1_get = find_where_it_fits((venti_available[0]+max_allocate_in_one_step)/venti_required[0], max_states, increment)
    where_state_2_get = find_where_it_fits((venti_available[1]+max_allocate_in_one_step)/venti_required[1], max_states, increment)
    where_state_3_get = find_where_it_fits((venti_available[2]+max_allocate_in_one_step)/venti_required[2], max_states, increment)
    where_state_4_get = find_where_it_fits((venti_available[3]+max_allocate_in_one_step)/venti_required[3], max_states, increment)
    reward_1_get = vals1[where_state_1_get]
    reward_2_get = vals2[where_state_2_get]
    reward_3_get = vals3[where_state_3_get]
    reward_4_get = vals4[where_state_4_get]
    reward_1_give = vals1[where_state_1_give]
    reward_2_give = vals2[where_state_2_give]
    reward_3_give = vals3[where_state_3_give]
    reward_4_give = vals4[where_state_4_give]
    reward_1_stay = vals1[where_state_1]
    reward_2_stay = vals2[where_state_2]
    reward_3_stay = vals3[where_state_3]
    reward_4_stay = vals4[where_state_4]
    reward_array_stay = [reward_1_stay, reward_2_stay, reward_3_stay, reward_4_stay]
    reward_array_give = [reward_1_give, reward_2_give, reward_3_give, reward_4_give]
    reward_array_get = [reward_1_get, reward_2_get, reward_3_get, reward_4_get]
    reward_array_get = np.asarray(reward_array_get)
    reward_array_give = np.asarray(reward_array_give)
    reward_array_stay = np.asarray(reward_array_stay)
    args_stay = np.argsort(reward_array_stay)
    args_give = np.argsort(reward_array_give)
    args_get= np.argsort(reward_array_get)
    print(args_get)
    state_get_1 = args_get[3]
    state_get_2 = args_get[2]
    state_give_1 = args_get[1]
    state_give_2 = args_get[0]
    state_to_state[state_give_1, state_get_1] = state_allocation_limit[state_give_1]
    state_to_state[state_give_2, state_get_2] = state_allocation_limit[state_give_2]
    return state_to_state
    

In [44]:
def optimal_policy_new(venti_available, venti_required, max_states, increment, max_allocate_in_one_step, n_states, vals1, vals2, vals3, vals4, state_allocation_limit):
    state_to_state = np.zeros((n_states,n_states)) # state_to_state transfer matrix
    state_state_1 = venti_available[0]/venti_required[0]
    state_state_2 = venti_available[1]/venti_required[1]
    state_state_3 = venti_available[2]/venti_required[2]
    state_state_4 = venti_available[3]/venti_required[3]
    where_state_1 = find_where_it_fits(state_state_1, max_states, increment)
    where_state_2 = find_where_it_fits(state_state_2, max_states, increment)
    where_state_3 = find_where_it_fits(state_state_3, max_states, increment)
    where_state_4 = find_where_it_fits(state_state_4, max_states, increment)
    where_state_1_get = find_where_it_fits((venti_available[0]+max_allocate_in_one_step)/venti_required[0], max_states, increment)
    where_state_2_get = find_where_it_fits((venti_available[1]+max_allocate_in_one_step)/venti_required[1], max_states, increment)
    where_state_3_get = find_where_it_fits((venti_available[2]+max_allocate_in_one_step)/venti_required[2], max_states, increment)
    where_state_4_get = find_where_it_fits((venti_available[3]+max_allocate_in_one_step)/venti_required[3], max_states, increment)
    reward_1_get = vals1[where_state_1_get]
    reward_2_get = vals2[where_state_2_get]
    reward_3_get = vals3[where_state_3_get]
    reward_4_get = vals4[where_state_4_get]
    reward_array_get = [reward_1_get, reward_2_get, reward_3_get, reward_4_get]
    reward_array_get = np.asarray(reward_array_get)
    args_get= np.argsort(reward_array_get)
    print(args_get)
    
    vent_actual_available = []
    vent_actual_needed = []
    for i in range(0,len(venti_required)):
        vent_actual_available.append(venti_available[i] - venti_required[i])
        vent_actual_needed.append(venti_required[i] - venti_available[i])
    vent_actual_available = np.asarray(vent_actual_available)
    vent_actual_needed = np.asarray(vent_actual_needed)
    
    state_get = [-1,-1]
    state_give = [-1,-1]
    state_get = np.asarray(state_get)
    state_give = np.asarray(state_give)
    
    flag = 0
    run = 0
    while flag < 2 and run < 4:
        for arg in args_get:
            run = run + 1
            if vent_actual_needed[arg] > 0 and flag < 2:
                state_get[flag] = arg
                flag = flag + 1
    
    flag1 = 0
    run1 = 0
    while flag1 < 2 and run1 < 4:
        for arg in args_get:
            if arg not in state_get:
                run1 = run1 + 1
                if vent_actual_available[arg] > 0 and flag1 < 2:
                    state_give[flag1] = arg
                    flag1 = flag1 + 1
        
    for i in range(0, len(state_get)):
        for j in range(0, len(state_give)):
            if state_get[i] != -1 and state_give[j] != -1:
                state_to_state[state_give[j], state_get[i]] = state_allocation_limit[state_give[j]]

    print(state_get)
    print(state_give)
    return state_to_state
    

In [112]:
def optimal_policy_fed(venti_available, venti_required, max_states, increment, max_allocate_in_one_step, n_states, vals1, vals2, vals3, vals4, state_allocation_limit, fed_pile, fed_pile_limit):
    state_to_state = np.zeros((n_states+1,n_states+1)) # state_to_state transfer matrix
    state_state_1 = venti_available[0]/venti_required[0]
    state_state_2 = venti_available[1]/venti_required[1]
    state_state_3 = venti_available[2]/venti_required[2]
    state_state_4 = venti_available[3]/venti_required[3]
    where_state_1 = find_where_it_fits(state_state_1, max_states, increment)
    where_state_2 = find_where_it_fits(state_state_2, max_states, increment)
    where_state_3 = find_where_it_fits(state_state_3, max_states, increment)
    where_state_4 = find_where_it_fits(state_state_4, max_states, increment)
    where_state_1_get = find_where_it_fits((venti_available[0]+max_allocate_in_one_step)/venti_required[0], max_states, increment)
    where_state_2_get = find_where_it_fits((venti_available[1]+max_allocate_in_one_step)/venti_required[1], max_states, increment)
    where_state_3_get = find_where_it_fits((venti_available[2]+max_allocate_in_one_step)/venti_required[2], max_states, increment)
    where_state_4_get = find_where_it_fits((venti_available[3]+max_allocate_in_one_step)/venti_required[3], max_states, increment)
    reward_1_get = vals1[where_state_1_get]
    reward_2_get = vals2[where_state_2_get]
    reward_3_get = vals3[where_state_3_get]
    reward_4_get = vals4[where_state_4_get]
    reward_array_get = [reward_1_get, reward_2_get, reward_3_get, reward_4_get]
    reward_array_get = np.asarray(reward_array_get)
    args_get= np.argsort(reward_array_get)
    print(args_get)
    
    vent_actual_available = []
    vent_actual_needed = []
    for i in range(0,len(venti_required)):
        vent_actual_available.append(venti_available[i] - venti_required[i])
        vent_actual_needed.append(venti_required[i] - venti_available[i])
    vent_actual_available = np.asarray(vent_actual_available)
    vent_actual_needed = np.asarray(vent_actual_needed)
    
    state_get = [-1,-1]
    state_give = [-1,-1]
    state_get = np.asarray(state_get)
    state_give = np.asarray(state_give)
    
    flag = 0
    run = 0
    while flag < 2 and run < 4:
        for arg in args_get:
            run = run + 1
            if vent_actual_needed[arg] > 0 and flag < 2:
                state_get[flag] = arg
                flag = flag + 1
    
    flag1 = 0
    run1 = 0
    while flag1 < 2 and run1 < 4:
        for arg in args_get:
            if arg not in state_get:
                run1 = run1 + 1
                if vent_actual_available[arg] > 0 and flag1 < 2:
                    state_give[flag1] = arg
                    flag1 = flag1 + 1
        
    for i in range(0, len(state_get)):
        fed_flag = 0
        for j in range(0, len(state_give)):
            if state_get[i] != -1 and state_give[j] != -1:
                state_to_state[state_give[j], state_get[i]] = state_allocation_limit[state_give[j]]
                if fed_flag == 0 and fed_pile > fed_pile_limit:
                    state_to_state[n_states, state_get[i]] = state_to_state[n_states, state_get[i]] + fed_pile_limit
                    fed_pile = fed_pile - fed_pile_limit
                    fed_flag = 1
                elif fed_flag == 0 and fed_pile < fed_pile_limit:
                    state_to_state[n_states, state_get[i]] = state_to_state[n_states, state_get[i]] + fed_pile
                    fed_pile = 0
                    fed_flag = 1
                    

    print(state_get)
    print(state_give)
    return (state_to_state,fed_pile)
    

In [113]:
optimal_policy_fed([1500, 2000, 124, 120], [500, 500, 500, 500], 20, 0.1, max_allocate_in_one_step, 4, vals1, vals2, vals3, vals4, [100,50,25,100], 13000, 400)

[2 3 0 1]
[2 3]
[0 1]


(array([[  0.,   0., 100., 100.,   0.],
        [  0.,   0.,  50.,  50.,   0.],
        [  0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.],
        [  0.,   0., 400., 400.,   0.]]), 12200)

In [114]:
next_num1

array([1150, 9950,  350, 1300])

In [115]:
next_num2

array([ 650, 5600, 4900, 1400])

In [116]:
def q_learning(max_vent_requirement, max_allocate_in_one_step, n, ratio_step, reward):
    i = 0
    vent_steps = []
    while i < max_vent_requirement/max_allocate_in_one_step:
        vent_steps.append(i*max_allocate_in_one_step)
        i = i + 1
    vent_steps = np.asarray(vent_steps)
    i = 0
    ratio_steps = []
    while i < n:
        ratio_steps.append(i*ratio_step)
        i = i + 1
    ratio_steps = np.asarray(ratio_steps)
    values = np.zeros((n))
    q_func = np.zeros((n,3))
    delta = float('inf')
    while delta > threshold:
        delta = 0
        for j in range(0, n):
            v = values[j]
            if j == 0:
                reward_backward = reward[j]
                val_backward = values[j]
                reward_stay = reward[j]
                reward_forward = reward[j+1]
                val_backward = values[j]
                val_stay = values[j]
                val_forward = values[j+1]
            elif j == n-1:
                reward_backward = reward[j-1]
                reward_stay = reward[j]
                reward_forward = reward[j]
                val_backward = values[j-1]
                val_stay = values[j]
                val_forward = values[j]
            else:
                reward_backward = reward[j-1]
                reward_stay = reward[j]
                reward_forward = reward[j+1]
                val_backward = values[j-1]
                val_stay = values[j]
                val_forward = values[j+1]
            total_reward_backward = reward_backward + discount_factor*val_backward
            total_reward_forward = reward_forward + discount_factor*val_forward
            total_reward_stay = reward_stay + discount_factor*val_stay
            values[j] = max(total_reward_backward, total_reward_stay, total_reward_forward)
            rew_array = [total_reward_backward, total_reward_stay, total_reward_forward]
            rew_array = np.asarray(rew_array)
            rew_arg = np.argmax(rew_array)
            q_func[j,0] = total_reward_backward
            q_func[j,1] = total_reward_stay
            q_func[j,2] = total_reward_forward
            delta = max(delta, abs(v - values[j]))
    return(q_func)

In [117]:
q1 = q_learning(max_vent_requirement, max_allocate_in_one_step, n, ratio_step, reward_state1)
q2 = q_learning(max_vent_requirement, max_allocate_in_one_step, n, ratio_step, reward_state2)
q3 = q_learning(max_vent_requirement, max_allocate_in_one_step, n, ratio_step, reward_state3)
q4 = q_learning(max_vent_requirement, max_allocate_in_one_step, n, ratio_step, reward_state4)

In [118]:
def q_policy(venti_available, venti_required, max_states, increment, max_allocate_in_one_step, n_states, q1, q2, q3, q4, state_allocation_limit):
    state_to_state = np.zeros((n_states,n_states)) # state_to_state transfer matrix
    state_state_1 = venti_available[0]/venti_required[0]
    state_state_2 = venti_available[1]/venti_required[1]
    state_state_3 = venti_available[2]/venti_required[2]
    state_state_4 = venti_available[3]/venti_required[3]
    where_state_1 = find_where_it_fits(state_state_1, max_states, increment)
    where_state_2 = find_where_it_fits(state_state_2, max_states, increment)
    where_state_3 = find_where_it_fits(state_state_3, max_states, increment)
    where_state_4 = find_where_it_fits(state_state_4, max_states, increment)
    
    q1_array = [q1[where_state_1, 0], q1[where_state_1, 1], q1[where_state_1, 2]]
    q1_array = np.asarray(q1_array)
    q2_array = [q2[where_state_2, 0], q2[where_state_2, 1], q2[where_state_2, 2]]
    q2_array = np.asarray(q2_array)
    q3_array = [q3[where_state_3, 0], q3[where_state_3, 1], q3[where_state_3, 2]]
    q3_array = np.asarray(q3_array)
    q4_array = [q4[where_state_4, 0], q4[where_state_4, 1], q4[where_state_4, 2]]
    q4_array = np.asarray(q4_array)
    
    max_q1 = np.argmax(q1_array)
    max_q2 = np.argmax(q2_array)
    max_q3 = np.argmax(q3_array)
    max_q4 = np.argmax(q4_array)
    
    max_array = [max_q1, max_q2, max_q3, max_q4]
    
    givers = []
    geters = []
    stayers = []
    
    for i in range(0, len(max_array)):
        if max_array[i] == 0:
            givers.append(i)
        elif max_array[i] == 1:
            stayers.append(i)
        elif max_array[i] == 2:
            geters.append(i)
            
    if len(givers) == 0 or len(geters) == 0:
        return state_to_state
    elif len(givers) >= 1 and len(geters) >= 1 and len(givers) >= len(geters):
        for i in givers:
            for j in geters:
                state_to_state[i, j] = state_allocation_limit[i]
    elif len(givers) >= 1 and len(geters) >= 1 and len(givers) < len(geters):
        for j in geters:
            for i in givers:
                state_to_state[i, j] = state_allocation_limit[i]
                
    return state_to_state

In [122]:
def q_policy_fed(venti_available, venti_required, max_states, increment, max_allocate_in_one_step, n_states, q1, q2, q3, q4, state_allocation_limit, fed_pile, fed_pile_limit):
    state_to_state = np.zeros((n_states+1,n_states+1)) # state_to_state transfer matrix
    state_state_1 = venti_available[0]/venti_required[0]
    state_state_2 = venti_available[1]/venti_required[1]
    state_state_3 = venti_available[2]/venti_required[2]
    state_state_4 = venti_available[3]/venti_required[3]
    where_state_1 = find_where_it_fits(state_state_1, max_states, increment)
    where_state_2 = find_where_it_fits(state_state_2, max_states, increment)
    where_state_3 = find_where_it_fits(state_state_3, max_states, increment)
    where_state_4 = find_where_it_fits(state_state_4, max_states, increment)
    
    q1_array = [q1[where_state_1, 0], q1[where_state_1, 1], q1[where_state_1, 2]]
    q1_array = np.asarray(q1_array)
    q2_array = [q2[where_state_2, 0], q2[where_state_2, 1], q2[where_state_2, 2]]
    q2_array = np.asarray(q2_array)
    q3_array = [q3[where_state_3, 0], q3[where_state_3, 1], q3[where_state_3, 2]]
    q3_array = np.asarray(q3_array)
    q4_array = [q4[where_state_4, 0], q4[where_state_4, 1], q4[where_state_4, 2]]
    q4_array = np.asarray(q4_array)
    
    max_q1 = np.argmax(q1_array)
    max_q2 = np.argmax(q2_array)
    max_q3 = np.argmax(q3_array)
    max_q4 = np.argmax(q4_array)
    
    max_array = [max_q1, max_q2, max_q3, max_q4]
    
    givers = []
    geters = []
    stayers = []
    
    for i in range(0, len(max_array)):
        if max_array[i] == 0:
            givers.append(i)
        elif max_array[i] == 1:
            stayers.append(i)
        elif max_array[i] == 2:
            geters.append(i)
            
    if len(givers) == 0 or len(geters) == 0:
        return state_to_state
    elif len(givers) >= 1 and len(geters) >= 1 and len(givers) >= len(geters):
        for i in givers:
            for j in geters:
                fed_flag = 0
                state_to_state[i, j] = state_allocation_limit[i]
                if fed_flag == 0 and fed_pile > fed_pile_limit:
                    state_to_state[n_states, j] = state_to_state[n_states, j] + fed_pile_limit
                    fed_pile = fed_pile - fed_pile_limit
                    fed_flag = 1
                elif fed_flag == 0 and fed_pile < fed_pile_limit:
                    state_to_state[n_states, j] = state_to_state[n_states, j] + fed_pile
                    fed_pile = 0
                    fed_flag = 1
    elif len(givers) >= 1 and len(geters) >= 1 and len(givers) < len(geters):
        for j in geters:
            fed_flag = 0
            for i in givers:
                state_to_state[i, j] = state_allocation_limit[i]
                if fed_flag == 0 and fed_pile > fed_pile_limit:
                    state_to_state[n_states, j] = state_to_state[n_states, j] + fed_pile_limit
                    fed_pile = fed_pile - fed_pile_limit
                    fed_flag = 1
                elif fed_flag == 0 and fed_pile < fed_pile_limit:
                    state_to_state[n_states, j] = state_to_state[n_states, j] + fed_pile
                    fed_pile = 0
                    fed_flag = 1
                
    return (state_to_state, fed_pile)

In [123]:
q_policy_fed([1500, 2000, 124, 120], [500, 500, 500, 500], 20, 0.1, max_allocate_in_one_step, 4, q1, q2, q3, q4, [100,50,25,100], 13000, 400)

(array([[  0.,   0., 100., 100.,   0.],
        [  0.,   0.,  50.,  50.,   0.],
        [  0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.],
        [  0.,   0., 800., 800.,   0.]]), 11400)